In [ ]:
# only for cuda enabled laptop and desktop
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
import tensorflow as tf
tf.keras.utils.get_file('cats_and_dogs_filtered.zip',"https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip")

In [ ]:
import os
import zipfile #to handle zip file

local_zip = '/home/aarav/.keras/datasets/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
os.listdir('cats_and_dogs_filtered')

In [ ]:
os.listdir('cats_and_dogs_filtered/validation')

In [ ]:
os.listdir('cats_and_dogs_filtered/validation/cats')

In [ ]:
len(os.listdir('cats_and_dogs_filtered/validation/cats'))

In [ ]:
len(os.listdir('cats_and_dogs_filtered/validation/dogs'))

In [ ]:
len(os.listdir('cats_and_dogs_filtered/train/cats'))

In [ ]:
len(os.listdir('cats_and_dogs_filtered/train/dogs'))

In [ ]:
for fname in os.listdir('cats_and_dogs_filtered/train/dogs'):
  print(fname)

In [ ]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img

In [ ]:
image_1 = load_img('cats_and_dogs_filtered/train/dogs/dog.131.jpg')

In [ ]:
image_1

In [ ]:
image_2 = load_img('cats_and_dogs_filtered/validation/cats/cat.2352.jpg')

In [ ]:
image_2

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = os.path.join('cats_and_dogs_filtered')

In [ ]:
path

In [ ]:
train_dir = os.path.join(path,'train')

In [ ]:
train_dir

In [ ]:
validation_dir = os.path.join(path,'validation')

In [ ]:
validation_dir

In [ ]:
train_cats_dir = os.path.join(train_dir,'cats')
train_dogs_dir = os.path.join(train_dir,'dogs')

In [ ]:
train_cats_dir

In [ ]:
train_dogs_dir

In [ ]:
validation_cats_dir = os.path.join(validation_dir,'cats')
validation_dogs_dir = os.path.join(validation_dir,'dogs')

In [ ]:
validation_cats_dir

In [ ]:
validation_dogs_dir

In [ ]:
os.listdir(validation_dogs_dir)

In [ ]:
type(os.listdir(validation_dogs_dir))

In [ ]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))
num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))
print('Cat training data ',num_cats_tr)
print('Dog training Data ',num_dogs_tr)
print('Cat Valid data ',num_cats_val)
print('Dog valid data ',num_dogs_val)

In [ ]:
total_train=num_cats_tr+num_dogs_tr
total_val=num_cats_val+num_dogs_val
print('Total Training images',total_train)
print('Total validation images',total_val)

In [ ]:
batch_size = 200
epochs = 25
IMG_HIEGHT = 150
IMG_WIDHT = 150

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HIEGHT,IMG_WIDHT),
                                                           class_mode='binary')

In [ ]:
train_data_gen.class_indices

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HIEGHT,IMG_WIDHT),
                                                              class_mode='binary')

In [ ]:
sample_train_images, label = next(train_data_gen)

In [ ]:
label

In [ ]:
sample_train_images.shape

In [ ]:
sample_train_images[0]

In [ ]:
plt.imshow(sample_train_images[0])

In [ ]:
sample_train_images[45].shape

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1,5,figsize=(20,20))
    axes = axes.flatten()
    for img,ax in zip(images_arr,axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_train_images[24:29])

In [ ]:
model = Sequential([
    Conv2D(16,3,padding='same',activation='relu',input_shape=(IMG_HIEGHT,IMG_WIDHT,3)),
    MaxPooling2D(),
    Conv2D(32,3,padding='same',activation='relu'),
    MaxPooling2D(),
    Conv2D(64,3,padding='same',activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
history = model.fit(train_data_gen,steps_per_epoch=total_train//batch_size,epochs=epochs,
                             validation_data=val_data_gen,validation_steps=total_val//batch_size)

In [ ]:
history.history

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(epochs_range,acc,label='Training Accuracy')
plt.plot(epochs_range,val_acc,label='Val Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy')
plt.subplot(1,2,2)
plt.plot(epochs_range,loss,label='Training Loss')
plt.plot(epochs_range,val_loss,label='Val LOss')
plt.legend(loc=0)
plt.title('Loss')
plt.show()

In [ ]:
# deal with overfitting

In [ ]:
#data augmentation
image_gen_train = ImageDataGenerator(rescale=1./255,
                                    rotation_range=45,
                                    width_shift_range=.15,
                                    height_shift_range=.15,
                                    horizontal_flip=True,
                                    zoom_range=0.5)

In [ ]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                    directory=train_dir,
                                                    shuffle=True,
                                                    target_size=(IMG_HIEGHT,IMG_HIEGHT),
                                                    class_mode='binary')

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                directory=validation_dir,
                                                target_size=(IMG_HIEGHT,IMG_WIDHT),
                                                class_mode='binary')

In [ ]:
model_new = Sequential([
    Conv2D(16,3,padding='same',activation='relu',input_shape=(IMG_HIEGHT,IMG_WIDHT,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32,3,padding='same',activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(64,3,padding='same',activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512,activation='relu'),
    Dropout(0.2),
    Dense(1,activation='sigmoid')
])

In [ ]:
model_new.summary()

In [ ]:
epochs = 100

In [ ]:
model_new.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model_new.fit(train_data_gen,steps_per_epoch=total_train//batch_size,
                                 epochs=epochs,
                                 validation_data=val_data_gen,
                                 validation_steps=total_val//batch_size)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(epochs_range,acc,label='Training Accuracy')
plt.plot(epochs_range,val_acc,label='Val Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy')
plt.subplot(1,2,2)
plt.plot(epochs_range,loss,label='Training Loss')
plt.plot(epochs_range,val_loss,label='Val LOss')
plt.legend(loc='upper right')
plt.title('Loss')
plt.show()

In [ ]:
import tensorflow as tf

In [ ]:
# save model

In [ ]:
tf.keras.models.save_model(model_new,'cat_dog.h5')